In [ ]:
import tensorflow as tf
import math

# Softmax Regression

## Model Setting

In [ ]:
# input, answer
x = tf.placeholder(dtype = tf.float64, shape=[None, 28 * 28], name = 'input')
y_ = tf.placeholder(dtype = tf.float64, shape=[None, 10], name = 'correct')

# weight and bias
w = tf.Variable(tf.zeros([28 * 28, 10], dtype =tf.float64), name = 'weight')
b = tf.Variable(tf.zeros([10], dtype=tf.float64), name = 'bias', dtype=tf.float64)

# prediction
y = tf.nn.softmax(tf.matmul(x, w) + b)

# cross entropy
# sum of gradient for each training data. reduction_indices = [1]
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), 1)) 


## load Input

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## training

In [ ]:
learning_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

# training loop
for i in range(50):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_:batch[1]})


# Model Evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

sess.close()

----

# Fully connected feedforward network

## Model Setting

In [ ]:
x = tf.placeholder(dtype = tf.float64, shape = [None, 28*28], name = 'input')
y_ = tf.placeholder(dtype = tf.float64, shape = [None, 10], name = 'observation')


In [ ]:
# layer 1 - 100 neurons
w1 = tf.Variable(tf.truncated_normal([28 * 28, 100], stddev=1.0/math.sqrt(float(28*28)), dtype=tf.float64))
b1 = tf.Variable(tf.zeros([100], dtype = tf.float64))
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

# layer 2 - 50 neurones
w2 = tf.Variable(tf.truncated_normal([100, 50], stddev=1.0/math.sqrt(float(100)), dtype=tf.float64))
b2 = tf.Variable(tf.zeros([50], dtype = tf.float64))
l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)

# layer 3 - 10 neurons. softmax / output layer
w3 = tf.Variable(tf.truncated_normal([50, 10], stddev=1.0/math.sqrt(float(50)), dtype = tf.float64))
b3 = tf.Variable(tf.zeros([10], dtype = tf.float64))
y = tf.nn.softmax(tf.nn.sigmoid(tf.matmul(l2, w3) + b3))
#y = tf.matmul(l2, w3) + b3

In [ ]:
# Error
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=1))
#cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, y_)
#loss = tf.reduce_mean(cross_entropy, reduction_indices=1)

In [ ]:
print(y)

In [ ]:
print(y_)

In [ ]:
# Training Setting
#train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

## Loading input data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})
    #sess.run([train_step], feed_dict={x:batch[0], y_:batch[1]})


In [ ]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))


In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64), reduction_indices=0)


In [ ]:
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))


In [ ]:
sess.close()

----

# CNN

## Network modeling - define graph

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
x = tf.placeholder(shape=[None, 28 * 28], dtype=tf.float32, name='input')
y_ = tf.placeholder(shape=[None, 10], dtype=tf.float32, name='label')
keep_prb = tf.placeholder(dtype=tf.float32)

In [ ]:
with tf.name_scope('hidden1'):
    filter_shape = [5, 5, 1, 32]
    w = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
    b = tf.Variable(tf.truncated_normal([32], stddev=0.1)) # why not [14, 14, 32]
    x_img = tf.reshape(x, [-1, 28, 28, 1])
    l1 = tf.nn.relu(tf.nn.conv2d(x_img, w, [1, 1, 1, 1], 'SAME') + b)
    l1 = tf.clip_by_value(l1, 0.0, 10.0)
    p1 = tf.nn.max_pool(l1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')

with tf.name_scope('hidden2'):
    filter_shape = [5, 5, 32, 64]
    w = tf.Variable(tf.truncated_normal(filter_shape, stddev = 0.1))
    b = tf.Variable(tf.truncated_normal([64], stddev=0.1))
    l2 = tf.nn.relu(tf.nn.conv2d(p1, w, [1, 1, 1, 1], 'SAME') + b)
    l2 = tf.clip_by_value(l2, 0.0, 10.0)
    p2 = tf.nn.max_pool(l2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')

with tf.name_scope('fully-connected'):
    w = tf.Variable(tf.truncated_normal([64 * 7 * 7, 128], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([128]))
    l2_flat = tf.reshape(p2, [-1, 64 * 7 * 7])
    l3 = tf.nn.relu(tf.matmul(l2_flat, w) + b)
    l3 = tf.clip_by_value(l3, 0.0, 10.0)

with tf.name_scope('softmax'):
    w = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([10]))
    y = tf.nn.softmax(tf.matmul(tf.nn.dropout(l3, keep_prb), w) + b)
    
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=1), reduction_indices=0)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1)), tf.float32))
#training_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
training_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


In [ ]:
sess = tf.InteractiveSession()

In [ ]:
sess.run(tf.initialize_all_variables())
test_img = mnist.test.images
test_lbl = mnist.test.labels
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if (i % 100 == 0):
        acc = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prb:0.5})
        cross_ent = cross_entropy.eval(feed_dict={x:batch[0], y_:batch[1]})
        print("Step: %d, training - cross entropy: %f, accuracy: %f" %(i, cross_ent, acc))
    sess.run([training_step], feed_dict={x:batch[0], y_:batch[1]})

acc = accuracy.eval(feed_dict={x:test_img, y_:test_lbl, keep_prb:1.0})
cross_ent = cross_entropy.eval(feed_dict={x:test_img, y_:test_lbl, keep_prb:1.0})
print("Step: %d, test - cross entropy: %f, accuracy: %f" %(i, cross_ent, acc))
    

In [ ]:
sess.close()

## 실험 결과

### Adam Optimizer와 GradientDescentOptimizer

이론적인 차이는 찾아봐야겠지만, 실제로 돌렸을 때 gradient descent는 gradient가 nan이 되는 경우가 있는데, weight를 적당히 clipping해 주면 learning을 돌릴 수는 있다.

Adam Optimizer의 성능이 훨씬 좋은 것 같다. 단 learning rate parameter가 동일하지는 않다.

### 성능 지표

```
[5x5] convolution, same padding, 32 output, relu, cliping(0,10), [2x2] max pooling
[5x5] convolution, same padding, 64 output, relu, cliping(0,10), [2x2] max pooling
fully connected, 128 output, relu, cliping(0,10)
softmax, drop out, keep prob 0.9
Adam(1e-4)

Step: 0, training - cross entropy: 3.272329, accuracy: 0.160000
Step: 100, training - cross entropy: 0.897650, accuracy: 0.700000
Step: 200, training - cross entropy: 0.478571, accuracy: 0.860000
Step: 300, training - cross entropy: 0.412178, accuracy: 0.840000
Step: 400, training - cross entropy: 0.309230, accuracy: 0.900000
Step: 500, training - cross entropy: 0.283316, accuracy: 0.940000
Step: 600, training - cross entropy: 0.110163, accuracy: 1.000000
Step: 700, training - cross entropy: 0.167554, accuracy: 0.980000
Step: 800, training - cross entropy: 0.254424, accuracy: 0.920000
Step: 900, training - cross entropy: 0.336519, accuracy: 0.880000
Step: 999, test - cross entropy: 0.157881, accuracy: 0.954300
```

```
[5x5] convolution, same padding, 32 output, relu, cliping(0,10), [2x2] max pooling
[5x5] convolution, same padding, 64 output, relu, cliping(0,10), [2x2] max pooling
fully connected, 128 output, relu, cliping(0,10)
softmax, drop out
Adam(1e-4)

Step: 0, training - cross entropy: 2.676068, accuracy: 0.120000
Step: 100, training - cross entropy: 0.716845, accuracy: 0.860000
Step: 200, training - cross entropy: 0.443431, accuracy: 0.880000
Step: 300, training - cross entropy: 0.226923, accuracy: 0.900000
Step: 400, training - cross entropy: 0.233747, accuracy: 0.940000
Step: 500, training - cross entropy: 0.267062, accuracy: 0.920000
Step: 600, training - cross entropy: 0.097137, accuracy: 0.980000
Step: 700, training - cross entropy: 0.181540, accuracy: 0.960000
Step: 800, training - cross entropy: 0.130207, accuracy: 0.980000
Step: 900, training - cross entropy: 0.105612, accuracy: 0.980000
Step: 999, test - cross entropy: 0.132037, accuracy: 0.960200
```

### 배운 것

fully connected 와 달리

1. weight matrix가 convolution patch이다. F.C는 [ output neuron x input neuron ] 였다면 CNN은 [patch w, patch h, in-channel, out-channel] 이 weight matrix가 된다.
2. input과 weight를 matrix multiplication 하는 대신 fd.nn.conv2d 연산을 수행한다. convolution을 곱하기라고 생각해도 된다
3. output이 1차원이 아니므로 bias도 [ output.w, output.h, output channel ] 이렇게 생겼을 것 같지만 그렇지 않다. [ output channel ] 과 같은 1차원 벡터다
4. input을 conv layer로 보낼때, conv layer에서 fully connected layer로 넘어갈 때 reshape, flat 해 줄 것
5. drop out은 마지막 softmax할 때. evaluate할 때는 keep prob를 1.0으로

----

## 연습장

In [ ]:
test_x = [[1, 1, 1], [1, 1, 1]]
print(tf.reduce_sum(test_x))
print(test_x)
print(tf.reduce_sum(test_x, 1))

In [ ]:
# tensor와 value
sess = tf.InteractiveSession()
sum_val = sess.run(tf.reduce_sum(test_x, 0))
sess.close()
print(sum_val)
print(tf.reduce_sum(test_x, 0))

In [ ]:
print(w[:,0])
print(w[0, :])
print(w[0, 0])

In [ ]:
print(tf.zeros([784, 10]))

In [ ]:
import numpy as np

In [ ]:
np.zeros([784, 10])

In [ ]:
# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

In [ ]:
print(product)
sess = tf.Session()
result = sess.run(product)
print(result)
sess.close()

In [ ]:
with tf.Session() as s:
    result = s.run(product)
    print(result)

In [ ]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))


In [ ]:
input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

# output:
# [array([ 21.], dtype=float32), array([ 7.], dtype=float32)]